# ImageNet Validation with Microcontroller Tutorial (CLI version)

> [ImageNet Validation with Microcontroller Tutorial](https://github.com/erectbranch/mcu-imagenet-tutorial)

---

## 1. Getting the configuration, Importing the libraries

In [ ]:
import os
import yaml
import numpy as np

In [ ]:
from utils.configs import SamplingConfig, CliConfig, yaml_to_config

with open('./configs/sampling.yaml') as f:
    config = yaml.safe_load(f)
sampling_config = yaml_to_config(config, SamplingConfig)

with open('./configs/cli_command.yaml') as f:
    config = yaml.safe_load(f)
cli_config = yaml_to_config(config, CliConfig)

In [ ]:
TOTAL_TRY      = sampling_config.sampling.num_subset
DATA_SIZE      = sampling_config.sampling.subset_size
NUM_SUBSET     = sampling_config.sampling.num_subset
IMAGENET_PATH  = sampling_config.sampling.save_path

SERIAL_PORT    = cli_config.stedgeai.serial_port
STM_AI_PATH    = os.path.expanduser(cli_config.stedgeai.path)
TEMP_WORKSPACE = os.path.expanduser(cli_config.stedgeai.temp)
OUTPUT_PATH    = os.path.expanduser(cli_config.stedgeai.report_path)

MODEL_PATH     = os.path.expanduser(cli_config.target.path)
TARGET_STM     = cli_config.target.mcu

---

## 2. Select a validation subset

In [ ]:
selected_idx = input("Enter the subset index of the validation data to inference.:}")

selected_idx = int(selected_idx)
assert 0 < selected_idx <= NUM_SUBSET, f"The input value must be an integer of {NUM_SUBSET} or less."
print(f'validation data subset has been selected.: {selected_idx} / {NUM_SUBSET}')

In [ ]:
completed     = os.listdir(OUTPUT_PATH)
current_try   = 1

for i in range(TOTAL_TRY):
    if str(i+1) not in completed:
        os.makedirs(f"{OUTPUT_PATH}/{i+1}/", exist_ok=True)
        current_try = i+1
        break

print(f"Inference in progress...: {current_try} / {TOTAL_TRY}")

In [ ]:
# (Optional) If you want to visualize the input data, you can use the following code.

import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

plt.rcParams["figure.figsize"] = (15,15)
current_inputs = np.load(f'{IMAGENET_PATH}/{selected_idx}_input.npy')

fig = plt.figure()
fig_cols = 5

for i, image in enumerate(current_inputs):
    current_loc = [(DATA_SIZE // fig_cols), fig_cols, (i + 1)]
    ax = fig.add_subplot(*current_loc)
    ax.imshow(image.squeeze(axis=0))
    ax.set_title(f'{i + 1}')
    ax.axis('off')
    
plt.show()

---

## 3. Inference on MCU

> please compare the `run_command` below to your IDE 'validate on target' log and adjust the `run_command` accordingly.

In [ ]:
# example code for inference
run_command = [f"arch -arm64 {STM_AI_PATH} validate --target {TARGET_STM} --name network -m {MODEL_PATH} ",     # 문장 마지막 공백 주의
                f"--compression none --verbosity 1 --allocate-inputs --allocate-outputs ",
                f"--workspace {TEMP_WORKSPACE} --output {OUTPUT_PATH}/{current_try} ",
                f"--valoutput {IMAGENET_PATH}/{selected_idx}_target.npy --mode target --valinput {IMAGENET_PATH}/{selected_idx}_input.npy ",
                f"--desc {SERIAL_PORT} --classifier"]
print(f'Start inferencing... (run command: {run_command})')

os.system(''.join(run_command))

---

## 4. Calculate the accuracy

In [ ]:
# Loading inference results
out = np.load(f'{OUTPUT_PATH}/{current_try}/network_val_io.npz')
mcu_outputs = out['c_outputs_1']

# Load the target labels
labels = np.load(f'{IMAGENET_PATH}/{selected_idx}_target.npy')

In [ ]:
def top_k_categorical_accuracy(y_true, y_pred_proba, k=1):
    return np.equal(np.argsort(y_pred_proba)[:, -k:], y_true[:, None]).any(axis=1).mean()

In [ ]:
top1_acc_list = []
top5_acc_list = []

for label, output in zip(labels, mcu_outputs):
    output = np.squeeze(output, axis=0)
    label_idx = np.argmax(label)
    label_idx = np.array([label_idx])

    top1 = top_k_categorical_accuracy(label_idx, output, k=1)
    top5 = top_k_categorical_accuracy(label_idx, output, k=5)
    
    top1_acc_list.append(top1 * 100)
    top5_acc_list.append(top5 * 100)

print(f'Top-1 average accuracy over {DATA_SIZE} images of validation data: {np.mean(top1_acc_list):.2f}%')
print(f'Top-5 average accuracy over {DATA_SIZE} images of validation data: {np.mean(top5_acc_list):.2f}%')

---